In [ ]:
import pandas as pd
import sys
import os
sys.path.append(os.path.abspath('../code'))
import config
from extractor import Extractor 

stparkpap_pap_en_name = "-".join(config.scrapeTarget["languages"])
stparkpap_pap_en = pd.read_csv(f"../data/stparkpap/{stparkpap_pap_en_name}.csv", na_filter=False)
stparkpap_pap_en = stparkpap_pap_en[["pap-simple", "en-simple"]]

In [ ]:
source_sentences = stparkpap_pap_en["pap-simple"].values
target_sentences = stparkpap_pap_en["en-simple"].values
extractor = Extractor()
word_translations = extractor.getWordTranslationsFromSentences(source_sentences, target_sentences)
# print(word_translations)

In [ ]:
from loader import Loader
from spellcheck import Spellcheck

load = Loader()
lan = "PAP-EN"
dictionary, dictionary_corpus = load.loadDictionary(lan)
# print(dictionary)
spell = Spellcheck(spellchecker_corpus=dictionary_corpus, lan=lan, load=True)

new_associations = {}

for k, v in word_translations.items():
    translation_exists = spell.trie.find(k.lower())
    if translation_exists or k.lower() in config.word_extraction_ignore["stparkpap"]:
        print(f"Skipping {k}")
    else:
        new_associations[k] = list(v.keys())[0]

print(f"Found: {len(new_associations)} Skipped: {len(word_translations)-len(new_associations)} Total: {len(word_translations)}")

In [ ]:
new_associations
new_associations_df = pd.DataFrame(list(new_associations.items()), columns=['pap-simple', 'en-simple'])
new_associations_df.to_csv("../data/stparkpap/pap-en(extracted_words).csv")